# Investigating High Traffic Indicators on Westbound I-94

This project takes a deeper analysis of the traffic patterns of interstate 94 in connection to some major factors, such as:
* Weather types and conditions, including the amount of rain, cloud, and snow
* Time of day, time of the week, etc.
* Temperature
* U.S. Holidays

The ***I-94*** is one of the longest U.S. interstate, streching ***1,585 miles*** along ***east-west*** from Billings, Montana, to Port Huron, Michigan. In deciding the project's **purpose**, we took into consideration the following factors that make I-94's traffic analysis particularly valuable for improving the overall safety and efficiency in the highway: 
* I-94 connects to near ***seven major cities***, including Minneapolis-St. Paul in Minnesota, Milwaukee in Wisconsin, and Chicago in Illinois, making it a vital transportation artery in the Midwest.
* It's economic impact on faciliatating the ***movement of goods*** and people between major population centers, industrial hubs, and distribution centers. 
* Notorious traffic volume between the Twin Cities (Minneapolis and St. Paul).

A **use case** of the analysis could be better understanding of what a normal range for traffic volume looks like for I-94, more accurately predict heavy traffic conditions, and plan beforehand for such situations based on the indicators to avoid any mishaps.

# Finding the Right Dataset
For this analysis, we found  the [dataset](https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume) collected by John Hogue in the UC Irvine Machine Learning Repository that provides hourly ***Mineapolis to St. Paul*** traffic details from ***2012 - 2014***.

Because the dataset found only covers Mineapolis to St. Paul traffic going **east to west**, we will focus the findings of our analysis to **I-94 Westbound** traffic in order to avoid generalization. This segment of I-94 is also notoriously known for its high traffic volume. Therefore, the dataset seems to be a good fit without detering too far from the original goal.

<center><img src=".\images\i94_map.jpg" alt="I-94 Map" width="500" height="300"></center>

In [11]:
# Import the data
import pandas as pd

i94_traffic = pd.read_csv('Metro_Interstate_Traffic_Volume.csv', encoding='latin1')
i94_traffic.head(3)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767


In [12]:
i94_traffic.tail(3)

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
48201,NaN,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159
48202,NaN,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450
48203,NaN,282.12,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 23:00:00,954


In [8]:
i94_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              61 non-null     object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB


### Initial Observations

* ***48204*** entries and ***9*** columns.
* Type correction is needed for `date_time` for accurate time-based analysis. The rest of the data types seem consistent with the Hogue's [documentation](https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume) for the dataset.
* `holiday` has only ***61 non-null*** values where most of them seem to be ***NaN*** values at a glance. According to the original [dataset](https://archive.ics.uci.edu/dataset/492/metro+interstate+traffic+volume), there should have no missing data for this column. We'll be taking a closer look at the `holiday` column to identify the issue.

In [17]:
# Analyzing the holiday column for NaN values
i94_traffic['holiday'].value_counts(dropna=False)

holiday
NaN                          48143
Labor Day                        7
Thanksgiving Day                 6
Christmas Day                    6
New Years Day                    6
Martin Luther King Jr Day        6
Columbus Day                     5
Veterans Day                     5
Washingtons Birthday             5
Memorial Day                     5
Independence Day                 5
State Fair                       5
Name: count, dtype: int64

After checking the original [CSV file](https://archive.ics.uci.edu/static/public/492/metro+interstate+traffic+volume.zip) using `COUNTIF`, the fields where no holidays take place have exactly ***48143*** ***"None"*** entries. This explains why ***48143*** entries are `NaN` as `pandas` probably interepreted `None` as missing values while importing the dataset.

## Cleaning and Prepping the Dataset for a Meaninful Analysis

1. Replace the `NaN` missing values. In this case, it would be accurate to replace the NaN values with the string ***"None"***.
2. Correct the `dtype` for `date_time` column for meaningful time-based analysis.
3. Check for any outliers in the *numeric* columns or unexpected values in the *categorical* columns.

### 1. Replace the NaN missing values with "None":

In [26]:
# Replace the missing values in the holiday column with "None"
i94_traffic['holiday'] = i94_traffic['holiday'].fillna("None")

# Validate changes
i94_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              48204 non-null  object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB


All ***48204*** entries are non-null.

### 2. Convert the date column to a datetime object for meaningful time-based analysis:

In [33]:
i94_traffic['date_time'].dtype

dtype('O')

In [48]:
# Convert the date_time column from 'object' to a 'datetime' object
i94_traffic['date_time'] = pd.to_datetime(i94_traffic['date_time'])

i94_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   holiday              48204 non-null  object        
 1   temp                 48204 non-null  float64       
 2   rain_1h              48204 non-null  float64       
 3   snow_1h              48204 non-null  float64       
 4   clouds_all           48204 non-null  int64         
 5   weather_main         48204 non-null  object        
 6   weather_description  48204 non-null  object        
 7   date_time            48204 non-null  datetime64[ns]
 8   traffic_volume       48204 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 3.3+ MB


`date_time` column is now of *datetime64[ns]* type.

### 3. Check for outliers or unexpected values:

